In [1]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB0"
# after writing commands a pause is needed
CMD_WRITE_PAUSE = 0.2

MM12_READ_INFOS   = b'\x55\x55\x00\x00\xaa'
MM12_READ_DISPLAY = b'\x55\x55\x01\x00\xab'

try:
    serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=2)

except serial.SerialException:
    print("Could not open serial port '{0}'!".format(SERIAL_PORT))
    raise

In [2]:
# function codes of Benning MM12 (table 6 in communication datasheet)
# the codes 0x32 ... 0x3E have not been included at the moment, because they are rarely used
# dictionary holds measuring modes and the corresponding SI base units
dict_function_codes_units = {
    0x00 : ('None', 'None'),
    0x01 : ('AC V', 'V'),
    0x02 : ('DC V', 'V'),
    0x03 : ('AC mV', 'mV'),
    0x04 : ('DC mV', 'mV'),
    0x05 : ('Ohm', 'Ohm'),
    0x06 : ('Continuity', 'Ohm'),
    0x07 : ('Diode', 'V'),
    0x08 : ('Capacitor', 'uF'),
    0x09 : ('AC A', 'A'),
    0x0A : ('DC A', 'A'),
    0x0B : ('AC mA', 'mA'),
    0x0C : ('DC mA', 'mA'),
    0x0D : ('°C', '°C'),
    0x0E : ('°F', '°F'),
    0x0F : ('Frequency', 'Hz'),
    0x10 : ('Duty', 'sec'),
    0x11 : ('Hz (V)', 'Hz'),
    0x12 : ('Hz (mV)', 'Hz'),
    0x13 : ('Hz (A)', 'Hz'),
    0x14 : ('Hz (mA)', 'Hz'),
    0x15 : ('AC+DC (V)', 'V'),
    0x16 : ('AC+DC (mV)', 'mV'),
    0x17 : ('AC+DC (A)', 'A'),
    0x18 : ('AC+DC (mA)', 'mA'),
    0x19 : ('LPF (V)', 'V'),
    0x1A : ('LPF (mV)', 'mV'),
    0x1B : ('LPF (A)', 'A'),
    0x1C : ('LPF (mA)', 'mA'),
    0x1D : ('AC uA', 'uA'),
    0x1E : ('DC uA', 'uA'),
    0x1F : ('DC A out', 'A'),
    0x20 : ('DC A out (Slow Linear)', 'A'),
    0x21 : ('DC A out (Fast Linear)', 'A'),
    0x22 : ('DC A out (Slow Step)', 'A'),
    0x23 : ('DC A out (Fast Step)', 'A'),
    0x24 : ('Loop Power', 'W'),
    0x25 : ('250 Ohm HART', 'Ohm'),
    0x26 : ('Voltage Sense', 'V'),
    0x27 : ('Peak Hold (V)', 'V'),
    0x28 : ('Peak Hold (mV)', 'mV'),
    0x29 : ('Peak Hold (A)', 'A'),
    0x2A : ('Peak Hold (mA)', 'mA'),
    0x2B : ('LoZ AC V', 'V'),
    0x2C : ('LoZ DC V', 'V'),
    0x2D : ('LoZ AC+DC (V)', 'V'),
    0x2E : ('LoZ LPF (V)', 'V'),
    0x2F : ('LoZ Hz (V)', 'V'),
    0x30 : ('LoZ Peak Hold (V)', 'V'),
    0x31 : ('Battery', '%')
}

#dict_function_codes_units

In [3]:
# range codes of Benning MM12 (table 7.1 and 7.2 in communication datasheet)
list_range_multiplier_ohm = {
    0x00 : 0.01,
    0x01 : 0.1,
    0x02 : 1,
    0x03 : 10,
    0x04 : 100,
    0x05 : 1000
}

list_range_multiplier_temp = {
    0x00 : 0.1
}

list_range_multiplier_voltage = {
    0x00 : 0.0001,
    0x01 : 0.001,
    0x02 : 0.01,
    0x03 : 0.1
}

# Attention: these hexadecimal values are actually implemented like this in the DMM firmware!
list_range_multiplier_LoZ_voltage = {
    0x02 : 0.01,
    0x03 : 0.1
}

list_range_multiplier_millivoltage = {
    0x00 : 0.001,
    0x01 : 0.01
}

# Attention: these hexadecimal values are actually implemented like this in the DMM firmware!
list_range_multiplier_current = {
    0x02 : 0.0001,
    0x03 : 0.001
}

list_range_multiplier_millicurrent = {
    0x00 : 0.001,
    0x01 : 0.01
}

list_range_multiplier_capacity = {
    0x00 : 0.00001,
    0x01 : 0.0001,
    0x02 : 0.001,
    0x03 : 0.01,
    0x04 : 0.1,
    0x05 : 1,
    0x06 : 10
}

list_range_multiplier_frequency = {
    0x00 : 0.01,
    0x01 : 0.1,
    0x02 : 1,
    0x03 : 10
}

list_range_multiplier_continuity = {
    0x00 : 0.01
}

list_range_multiplier_diode = {
    0x00 : 0.001
}

list_range_multiplier_NONE = {
    0x00 : 1
}

In [4]:
# scope codes of Benning MM12 (table 2 in communication datasheet)
# bits 7..3 represent the unit
list_scope_unit = {
    0x00 : 'None',
    0x01 : 'V',
    0x02 : 'mV',
    0x03 : 'A',
    0x04 : 'mA',
    0x05 : 'dB',
    0x06 : 'dBm',
    0x07 : 'mF',
    0x08 : 'uF',
    0x09 : 'nF',
    0x0A : 'GOhm',
    0x0B : 'MOhm',
    0x0C : 'kOhm',
    0x0D : 'Ohm',
    0x0E : '%',
    0x0F : 'MHz',
    0x10 : 'kHz',
    0x11 : 'Hz',
    0x12 : '°C',
    0x13 : '°F',
    0x14 : 'sec',
    0x15 : 'ms',
    0x16 : 'us',
    0x17 : 'ns',
    0x18 : 'uA',
    0x19 : 'min',
    0x1A : 'kW',
    0x1B : 'PF'
}

# bits 2..0 represent the multiplier
list_scope_multiplier = {
    0x00 : None,
    0x01 : 0.1,
    0x02 : 0.01,
    0x03 : 0.001,
    0x04 : 0.0001
}

In [5]:
def func_mask_MSB(byte_in):
    byte_out = byte_in & 0b01111111
    return byte_out

In [6]:
def func_convert_hex_str_human_readable(str_in):
    hex_string = str_in.hex()
    # fill string with spaces for better reading of the hex string
    hex_string_wSpaces = " ".join(hex_string[i-1:i+1] for i, c in enumerate(hex_string) if i%2)
    return hex_string_wSpaces

In [10]:
print(func_convert_hex_str_human_readable(MM12_READ_DISPLAY))

# flush input buffer, discarding all its contents
serial.reset_input_buffer()

serial.write(MM12_READ_DISPLAY)

55 55 01 00 ab


5

In [11]:
hex_string = serial.read(17)

print(func_convert_hex_str_human_readable(hex_string))

hex_bytes = hex_string.hex()

55 55 01 0c 08 83 f6 0c 00 42 00 00 00 70 00 00 f6


In [12]:
# byte 0 from data payload is the function code
func_code = hex_bytes[8:10]
# convert from hex to int
func_code_int = int(func_code, 16)

# mask the MSB that identifies the auto or manual test
func_code_int = func_mask_MSB(func_code_int)

func_unit_str = dict_function_codes_units[func_code_int]

print(func_unit_str)

('Capacitor', 'uF')


In [13]:
# byte 1 from data payload is the range code
range_code = hex_bytes[10:12]
# convert from hex to int
range_code_int = int(range_code, 16)

# mask the MSB that identifies the auto or manual range
range_code_int = func_mask_MSB(range_code_int)

# function: Ohm
if func_code_int == 0x05:
    range_float = list_range_multiplier_ohm[range_code_int]

# function: Temperature (°C or °F)
elif ( func_code_int == 0x0D or 
        func_code_int == 0x0E ):
    range_float = list_range_multiplier_temp[range_code_int]

# function: Voltage (AC V, LPF (V), Peak Hold (V), DC V, AC+DC (V))
elif ( func_code_int == 0x01 or
        func_code_int == 0x19 or
        func_code_int == 0x27 or
        func_code_int == 0x02 or
        func_code_int == 0x15 ):
    range_float = list_range_multiplier_voltage[range_code_int]

# function: Millivoltage (AC mV, LPF (mV), Peak Hold (mV), DC mV, AC+DC (mV))
elif ( func_code_int == 0x03 or
        func_code_int == 0x1A or
        func_code_int == 0x28 or
        func_code_int == 0x04 or
        func_code_int == 0x16 ):
    range_float = list_range_multiplier_millivoltage[range_code_int]

# function: LoZ Voltage (LoZ AC V, LoZ DC V)
elif ( func_code_int == 0x2B or
        func_code_int == 0x2C ):
    range_float = list_range_multiplier_LoZ_voltage[range_code_int]

# function: Current (AC A, DC A, AC+DC (A), LPF (A), Peak Hold (A))
elif ( func_code_int == 0x09 or
        func_code_int == 0x0A or
        func_code_int == 0x17 or
        func_code_int == 0x1B or
        func_code_int == 0x29 ):
    range_float = list_range_multiplier_current[range_code_int]

# function: Millicurrent (AC mA, DC mA, AC+DC (mA), LPF (mA), Peak Hold (mA))
elif ( func_code_int == 0x0B or
        func_code_int == 0x0C or
        func_code_int == 0x18 or
        func_code_int == 0x1C or
        func_code_int == 0x2A ):
    range_float = list_range_multiplier_millicurrent[range_code_int]

# function: Capacitor (in µF)
elif func_code_int == 0x08:
    range_float = list_range_multiplier_capacity[range_code_int]

# function: Frequency (Hz (V), Hz (mV), Hz (A), Hz (mA))
elif ( func_code_int == 0x11 or
        func_code_int == 0x12 or
        func_code_int == 0x13 or
        func_code_int == 0x14 ):
    range_float = list_range_multiplier_frequency[range_code_int]

# function: Continuity (Ohm)
elif func_code_int == 0x06:
    range_float = list_range_multiplier_continuity[range_code_int]

# function: Diode (V)
elif func_code_int == 0x07:
    range_float = list_range_multiplier_continuity[range_code_int]

# function: NONE
elif func_code_int == 0x00:
    range_float = list_range_multiplier_NONE[range_code_int]

print(range_float)

0.01


In [14]:
# bytes 2..4 from data payload are the measuring value (24bit signed integer)
disp_value_hex = bytes.fromhex(hex_bytes[12:18])
#disp_value_hex

# convert bytes to 24bit signed integer, LSB to MSB (little endian)
disp_value_sint = int.from_bytes(disp_value_hex, 'little', signed=True)
disp_value_sint

3318

In [15]:
print("Display value in mode '{}': {} {}".format(func_unit_str[0], disp_value_sint*range_float, func_unit_str[1]))

Display value in mode 'Capacitor': 33.18 uF


In [16]:
# byte 5 from data payload represent the scope code
scope_code_hex = hex_bytes[18:20]
# convert from hex to int
scope_code_int = int(scope_code_hex, 16)

# bits 7..3 represent the unit
unit_code = scope_code_int >> 3
unit_str = list_scope_unit[unit_code]
print(unit_str)

# bits 2..0 represent the multiplier
multiplier_code = scope_code_int & 0b00000111
multiplier_float = list_scope_multiplier[multiplier_code]
print(multiplier_float)

uF
0.01


In [17]:
print("Display value: {:.4f} {}".format(disp_value_sint*multiplier_float, unit_str))

Display value: 33.1800 uF


In [34]:
print(func_convert_hex_str_human_readable(MM12_READ_INFOS))

# flush input buffer, discarding all its contents
serial.reset_input_buffer()

serial.write(MM12_READ_INFOS)

55 55 00 00 aa


5

In [35]:
hex_string = serial.read(57)

print(func_convert_hex_str_human_readable(hex_string))

hex_bytes = hex_string.hex()

55 55 00 34 42 45 4e 4e 49 4e 47 20 4d 4d 31 32 20 20 20 20 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 20 20 20 20 20 20 20 20 32 38 36 30 30 30 38 32 06 00 73 00 8f


In [101]:
# bytes 0..31 from data payload represent the model name
model_name_hex = hex_bytes[8:72]

# convert hex string to bytes object
bytes_object = bytes.fromhex(model_name_hex)

# convert bytes object to ASCII representation
model_name_str = bytes_object.decode("ASCII")

model_name_str = model_name_str.replace("\x00", "")
model_name_str = model_name_str.strip()

print("Model name: {}".format(model_name_str))

Model name: BENNING MM12


In [100]:
# bytes 32..47 from data payload represent the serial number
serial_number_hex = hex_bytes[72:104]

# convert hex string to bytes object
bytes_object = bytes.fromhex(serial_number_hex)

# convert bytes object to ASCII representation
serial_number_str = bytes_object.decode("ASCII")

serial_number_str = serial_number_str.strip()

print("Serial number: {}".format(serial_number_str))

Serial number: 28600082


In [105]:
# bytes 48..49 from data payload represent the model ID
model_ID_hex = bytes.fromhex(hex_bytes[104:108])

# convert hex bytes to integer
model_ID_int = int.from_bytes(model_ID_hex, 'little', signed=False)
model_ID_int

print("Model ID: {}".format(model_ID_int))

Model ID: 6


In [99]:
# bytes 50..51 from data payload represent the firmware version
firmware_version_hex = bytes.fromhex(hex_bytes[108:112])

# convert hex bytes to integer
firmware_version_int = int.from_bytes(firmware_version_hex, 'little', signed=False)

print("Firmware version: {}".format(firmware_version_int/100))

Firmware version: 1.15
